In [100]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

In [121]:
if 'ygong' in os.getcwd():
    filepath = "../data"
    dir_out = "../output"
else:
    filepath = "/home/gong/Documents/Kaggle_July2019/data"
    dir_out = "/home/gong/Documents/Kaggle_July2019/output"

def load_data(filepath):
    train = pd.read_csv(os.path.join(filepath, 'train.csv'))
    test = pd.read_csv(os.path.join(filepath, 'test.csv'))
    submit = pd.read_csv(os.path.join(filepath, 'sample_submission.csv'))
    structures = pd.read_csv(os.path.join(filepath, 'structures.csv'))

    print('Train dataset shape is -> rows: {} cols:{}'.format(train.shape[0], train.shape[1]))
    print('Test dataset shape is  -> rows: {} cols:{}'.format(test.shape[0], test.shape[1]))
    print('Sample submission dataset shape is  -> rows: {} cols:{}'.format(submit.shape[0], submit.shape[1]))
    print('Structures dataset shape is  -> rows: {} cols:{}'.format(structures.shape[0], structures.shape[1]))
    print('\n')

    return train, test, submit, structures

train, test, submit, structures = load_data(filepath)

Train dataset shape is -> rows: 4658147 cols:6
Test dataset shape is  -> rows: 2505542 cols:5
Sample submission dataset shape is  -> rows: 2505542 cols:2
Structures dataset shape is  -> rows: 2358657 cols:6




In [122]:
# split number of interventing bonds and bond combination
# def extract_n_bonds(row):
#     return int(row['type'][0])
# def extract_type_bond(row):
#     return row['type'][2:]
# def check_covalent(row):
#     if 'JHH' in row['type']:
#         cov = 0
#     else:
#         cov = 1
#     return cov

# train = pd.concat([train, pd.DataFrame({'n_bonds': train.agg(extract_n_bonds, axis=1), 
#                                         'type_bond': train.agg(extract_type_bond, axis=1),
#                                         'cov':train.agg(check_covalent, axis=1)})], axis=1)
# test = pd.concat([test, pd.DataFrame({'n_bonds': test.agg(extract_n_bonds, axis=1), 
#                                       'type_bond': test.agg(extract_type_bond, axis=1),
#                                       'cov':test.agg(check_covalent, axis=1)})], axis=1)

In [123]:
# calculate bond length adopted from @Chanran Kim - Kaggle kernel
def map_atom_info(df, atom_idx):
    df = pd.merge(df, structures, how = 'left',
                  left_on  = ['molecule_name', f'atom_index_{atom_idx}'],
                  right_on = ['molecule_name',  'atom_index'])
    
    df = df.drop('atom_index', axis=1)
    df = df.rename(columns={'atom': f'atom_{atom_idx}',
                            'x': f'x_{atom_idx}',
                            'y': f'y_{atom_idx}',
                            'z': f'z_{atom_idx}'})
    return df

train = map_atom_info(train, 0)
train = map_atom_info(train, 1)

test = map_atom_info(test, 0)
test = map_atom_info(test, 1)

train_p_0 = train[['x_0', 'y_0', 'z_0']].values
train_p_1 = train[['x_1', 'y_1', 'z_1']].values
test_p_0 = test[['x_0', 'y_0', 'z_0']].values
test_p_1 = test[['x_1', 'y_1', 'z_1']].values

train['dist'] = np.linalg.norm(train_p_0 - train_p_1, axis=1)
test['dist'] = np.linalg.norm(test_p_0 - test_p_1, axis=1)

In [124]:
def onehot_features(df_data):
    onehot_list = ['type']
    for item in onehot_list:
        df_data = pd.concat([df_data, pd.get_dummies(df_data[item], 
                                                     prefix=item)], axis=1)
    drop_features = onehot_list + ['id', 'molecule_name', 'atom_0', 'atom_1']
    df_data.drop(columns=drop_features, inplace=True)
    return df_data
# select features for training
train = onehot_features(train)
test = onehot_features(test)

In [119]:
train.to_csv(os.path.join(dir_out, "_train_0730.csv"), index=False)
print("Saved training dataset to {}".format(os.path.join(dir_out, "_train.csv")))

test.to_csv(os.path.join(dir_out, "_test_0730.csv"), index=False)
print("Saved test dataset to {}".format(os.path.join(dir_out, "_test.csv")))

Saved training dataset to ../output/_train.csv
Saved test dataset to ../output/_test.csv


In [125]:
train.head()

,atom_index_0,atom_index_1,scalar_coupling_constant,x_0,y_0,z_0,x_1,y_1,z_1,dist,type_1JHC,type_1JHN,type_2JHC,type_2JHH,type_2JHN,type_3JHC,type_3JHH,type_3JHN
0,1,0,84.8076,0.002150,-0.006031,0.001976,-0.012698,1.085804,0.008001,1.091953,1,0,0,0,0,0,0,0
1,1,2,-11.2570,0.002150,-0.006031,0.001976,1.011731,1.463751,0.000277,1.783120,0,0,0,1,0,0,0,0
2,1,3,-11.2548,0.002150,-0.006031,0.001976,-0.540815,1.447527,-0.876644,1.783147,0,0,0,1,0,0,0,0
3,1,4,-11.2543,0.002150,-0.006031,0.001976,-0.523814,1.437933,0.906397,1.783157,0,0,0,1,0,0,0,0
4,2,0,84.8074,1.011731,1.463751,0.000277,-0.012698,1.085804,0.008001,1.091952,1,0,0,0,0,0,0,0


In [113]:
np.array([1,2,3])+np.array([0,0,1])

array([1, 2, 4])